<h1>Combining and Reshaping
Data</h1>

This chapter has two general categories of topics: combination and reshaping of data. The
first two sections will cover the capabilities provided by pandas to combine the data from
multiple pandas objects together. Combination of data in pandas is performed by
concatenating two sets of data, where data is combined simply along either axes but
without regard to relationships in the data. Or data can be combined using relationships in
the data by using a pandas capability referred to as merging, which provides join
operations that are similar to those in many relational databases.

The remaining sections will examine the three primary means reshaping data in pandas.
These will examine the processes of pivoting, stacking and unstacking, and melting of
data. Pivoting allows us to restructure pandas data similarly to how spreadsheets pivot
data by creating new index levels and moving data into columns based upon values (or
vice-versa). Stacking and unstacking are similar to pivoting, but allow us to pivot data
organized with multiple levels of indexes. And finally, melting allows us to restructure
data into unique ID-variable-measurement combinations that are or required for many
statistical analyses.

Specifically, in this chapter we will examine the following concepts of combining and
reshaping pandas data:
<ul>
    <li>Concatenation</li>
<li>Merging and joining</li>
<li>Pivots</li>
<li>Stacking/unstacking</li>
<li>Melting</li>
<li>The potential performance benefits of stacked data</li>

In [1]:
# import pandas, numpy and datetime
import numpy as np
import pandas as pd
import datetime

# Set some pandas options for controlling output
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

<h3>Concatenating Data</h3>

Concatenation in pandas is the process of either adding rows to the end of an existing
Series or DataFrame object or adding additional columns to a DataFrame. In pandas,
concatenation is performed via the pandas function pd.concat(). The function will
perform the operation on a specific axis and as we will see, will also perform any required
set logic involved in aligning along that axis.

In [2]:
# two Series objects to concatenate
s1 = pd.Series(np.arange(0, 3))
s2 = pd.Series(np.arange(5, 8))

In [3]:
# concatenate them
pd.concat([s1, s2])

0    0
1    1
2    2
0    5
1    6
2    7
dtype: int64

Two DataFrame objects can also be similarly concatenated.

In [4]:
# create two DataFrame objects to concatenate
# using the same index labels and column names,
# but different values

df1 = pd.DataFrame(np.arange(9).reshape(3, 3), 
                   columns=['a', 'b', 'c'])

In [5]:
df2 = pd.DataFrame(np.arange(9, 25).reshape(4, 4),
                   columns=['a', 'b', 'c', 'd'])

In [6]:
df1, df2

(   a  b  c
 0  0  1  2
 1  3  4  5
 2  6  7  8,
     a   b   c   d
 0   9  10  11  12
 1  13  14  15  16
 2  17  18  19  20
 3  21  22  23  24)

In [7]:
# do the concat
pd.concat([df1, df2])

    a   b   c     d
0   0   1   2   NaN
1   3   4   5   NaN
2   6   7   8   NaN
0   9  10  11  12.0
1  13  14  15  16.0
2  17  18  19  20.0
3  21  22  23  24.0

<p>The process of concatenating the two DataFrame objects will first identify the set of
columns formed by aligning the labels in the columns, effectively determining the union
of the column names. The resulting DataFrame object will then consist of that set of
columns, and columns with identical names will not be duplicated.</p>
<p>Rows will be then be added to the result, in the order of the each of the objects passed to
pd.concat(). If a column in the result does not exist in the object being copied, NaN
values will be filled in those locations. Duplicate row index labels can occur.</p>

In [8]:
# concat the two objects, but create an index using the
# given keys
c = pd.concat([df1, df2], keys=['df1', 'df2'])
# note the labeling of the rows in the output
c

        a   b   c     d
df1 0   0   1   2   NaN
    1   3   4   5   NaN
    2   6   7   8   NaN
df2 0   9  10  11  12.0
    1  13  14  15  16.0
    2  17  18  19  20.0
    3  21  22  23  24.0

In [9]:
# we can extract the data originating from
# the first or second source DataFrame
c.loc['df2']

    a   b   c     d
0   9  10  11  12.0
1  13  14  15  16.0
2  17  18  19  20.0
3  21  22  23  24.0

The pd.concat() function also allows you to specify the axis on which to apply the
concatenation. The following concatenates the two DataFrame objects along the columns
axis:

In [10]:
# concat df1 and df2 along columns
# aligns on row labels, has duplicate columns
pd.concat([df1, df2], axis=1)

     a    b    c   a   b   c   d
0  0.0  1.0  2.0   9  10  11  12
1  3.0  4.0  5.0  13  14  15  16
2  6.0  7.0  8.0  17  18  19  20
3  NaN  NaN  NaN  21  22  23  24

Note that the result now contains duplicate columns. The concatenation first aligns by the
row index labels of each DataFrame object, and then fills in the columns from the first
DataFrame object and then the second. The columns are not aligned and result in duplicate
values.

The same rules of alignment and filling of NaN values apply in this case, except that they
are applied to the rows’ index labels.

<p>A concatenation of two or more DataFrame objects actually performs an outer join
operation along the index labels on the axis opposite to the one specified. This makes the
result of the concatenation similar to having performed a union of those index labels, and
then data is filled based on the alignment of those labels to the source objects.</p>
<p>The type of join can be changed to an inner join and can be performed by specifying
join='inner' as the parameter. The inner join then logically performs an intersection
instead of a union. The following demonstrates this.</p>

In [11]:
# do an inner join instead of outer
# results in one row
df = pd.concat([df1, df2], axis=1, join='inner', keys=['df1','df2'])
df

  df1       df2            
    a  b  c   a   b   c   d
0   0  1  2   9  10  11  12
1   3  4  5  13  14  15  16
2   6  7  8  17  18  19  20

In [12]:
# VS Outer join
pd.concat([df1, df2], axis=1)

     a    b    c   a   b   c   d
0  0.0  1.0  2.0   9  10  11  12
1  3.0  4.0  5.0  13  14  15  16
2  6.0  7.0  8.0  17  18  19  20
3  NaN  NaN  NaN  21  22  23  24

In [13]:
# retrieve the data that originated from the
# DataFrame with key 'df2'
df['df2']

    a   b   c   d
0   9  10  11  12
1  13  14  15  16
2  17  18  19  20

A DataFrame (and Series) object also contains an .append() method, which will
concatenate the two specified DataFrame objects along the row index labels.

In [14]:
# append does a concatenate along axis=0
# duplicate row index labels can result
df1.append(df2)

    a   b   c     d
0   0   1   2   NaN
1   3   4   5   NaN
2   6   7   8   NaN
0   9  10  11  12.0
1  13  14  15  16.0
2  17  18  19  20.0
3  21  22  23  24.0

As with a concatenation on axis=1, the index labels in the rows are copied without
consideration of the creation of duplicates, and the columns labels are joined in a manner
which ensures no duplicate column name is included in the result. If you would like to
ensure that the resulting index does not have duplicates but preserves all of the rows, you
can use the ignore_index=True parameter. This essentially returns the same result except
with new Int64Index:

In [15]:
# remove duplicates in the result index by ignoring the
# index labels in the source DataFrame objects
df1.append(df2, ignore_index=True)

    a   b   c     d
0   0   1   2   NaN
1   3   4   5   NaN
2   6   7   8   NaN
3   9  10  11  12.0
4  13  14  15  16.0
5  17  18  19  20.0
6  21  22  23  24.0

<h3>Merging and joining data</h3>

pandas allows the merging of pandas objects with database-like join operations using the
pd.merge() function and the .merge() method of a DataFrame object. These joins are
high performance and are performed in memory. A merge combines the data of two
pandas objects by finding matching values in one or more columns or row indexes. It then
returns a new object that represents a combination of the data from both based on
relational-database-like join semantics applied to those values.

Merges are useful as they allow us to model a single DataFrame for each type of data (one
of the rules of having tidy data) but to be able to relate data in different DataFrame objects
using values existing in both sets of data.

<h4>An overview of merges</h4>

To demonstrate this in pandas, we will use the following two DataFrame
objects, where one represents a list of customer details, and the other represents the orders
made by customers and what day the order was made. They will be related to each other
using the CustomerID columns in each:

In [16]:
customers = {'CustomerID': [10, 11],
             'Name'      : ['Mike', 'Marcia'],
             'Address'   : ['Address for Mike', 'Address for Marcia']}

customers = pd.DataFrame(customers)
customers

   CustomerID    Name             Address
0          10    Mike    Address for Mike
1          11  Marcia  Address for Marcia

In [17]:
orders = {'CustomerID': [10, 11, 10],
          'OrderDate':  [datetime.date(2014, 12, 1),
                         datetime.date(2014, 12, 1),
                         datetime.date(2014, 12, 1)
                        ]
         }

orders = pd.DataFrame(orders)
orders

   CustomerID   OrderDate
0          10  2014-12-01
1          11  2014-12-01
2          10  2014-12-01

In [18]:
# merge customers and orders so we can ship the items
customers.merge(orders)

   CustomerID    Name             Address   OrderDate
0          10    Mike    Address for Mike  2014-12-01
1          10    Mike    Address for Mike  2014-12-01
2          11  Marcia  Address for Marcia  2014-12-01

<p>To be even more detailed, what pandas has specifically done is the following:</p>
<ol>
    <li>Determines the columns in both customers and orders with common labels. These
        columns are treated as the keys to perform the join.</li>
<li>It creates a new DataFrame whose columns are the labels from the keys identified in
step 1, followed by all of the non-key labels from both objects.</li>
<li>It matches values in the key columns of both DataFrame objects.</li>
<li>It then creates a row in the result for each set of matching labels.</li>
<li>It then copies the data from those matching rows from each source object into that
respective row and columns of the result.</li>
<li>It assigns a new Int64Index to the result.</li>

To explicitly specify which column use to relate the objects, use the on parameter. The
following performs a merge using only the values in the key1 column of both DataFrame
objects:

In [19]:
left_data = {'key1': ['a', 'b', 'c'],
             'key2': ['x', 'y', 'z'],
            'lval1': [ 0, 1, 2]}

right_data = {'key1': ['a', 'b', 'c'],
              'key2': ['x', 'a', 'z'],
              'rval1': [ 6, 7, 8 ]}

left = pd.DataFrame(left_data, index=[0, 1, 2])
right = pd.DataFrame(right_data, index=[1, 2, 3])
left, right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
   key1 key2  rval1
 1    a    x      6
 2    b    a      7
 3    c    z      8)

In [20]:
# demonstrate merge without specifying columns to merge
# this will implicitly merge on all common columns
left.merge(right)

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

In [21]:
# demonstrate merge using an explicit column
# on needs the value to be in both DataFrame objects
left.merge(right, on='key1')

  key1 key2_x  lval1 key2_y  rval1
0    a      x      0      x      6
1    b      y      1      a      7
2    c      z      2      z      8

<p>Comparing this result to the previous example, as only the values in the key1 column were
used to relate the data in the two objects, the result now has three rows as there are
matching a, b, and c values in that single column of both objects.</p>
<p>The on parameter can also be given a list of column names. The following reverts to using
both the key1 and key2 columns, resulting in being identical the earlier example where
those two columns where implicitly identified by pandas:</p>

In [22]:
# merge explicitly using two columns
left.merge(right, on=['key1', 'key2'])

  key1 key2  lval1  rval1
0    a    x      0      6
1    c    z      2      8

<p>The columns specified with on need to exist in both DataFrame objects. If you would like
to merge based on columns with different names in each object, you can use the left_on
and right_on parameters, passing the name or names of columns to each respective
parameter.</p><p>To perform a merge with the labels of the row indexes of the two DataFrame objects, use
the left_index=True and right_index=True parameters (both need to be specified):</p>

In [23]:
left,right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
   key1 key2  rval1
 1    a    x      6
 2    b    a      7
 3    c    z      8)

In [24]:
# join on the row indices of both matrices
pd.merge(left, right, left_index=True, right_index=True)

  key1_x key2_x  lval1 key1_y key2_y  rval1
1      b      y      1      a      x      6
2      c      z      2      b      a      7

<p>This has identified that the index labels in common are 1 and 2, so the resulting DataFrame
has two rows with these values and labels in the index. pandas then creates a column in
the result for every column in both objects and then copies the values.</p><p>
As both DataFrame objects had a column with an identical name, key, the columns in the
result have the _x and _y suffixes appended to them to identify the DataFrame they
originated from. _x is for left and _y for right. You can specify these suffixes using the
suffixes parameter and passing a two-item sequence.</p>

<h3>Specifying the join semantics of a merge operation</h3>

The default type of join performed by pd.merge() is an inner join. To use another join
method, the method of join to be used can be specified using the how parameter of the
pd.merge() function (or the .merge() method). The valid options are:

<ul>
    <li>inner: This is the intersection of keys from both DataFrame objects</li>
<li>outer: This is the union of keys from both DataFrame objects</li>
<li>left: This only uses keys from the left DataFrame</li>
<li>right: This only uses keys from the right DataFrame</li>
    </ul>

In [25]:
left,right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
   key1 key2  rval1
 1    a    x      6
 2    b    a      7
 3    c    z      8)

In [26]:
# outer join, merges all matched data,
# and fills unmatched items with NaN
left.merge(right, how='outer')

  key1 key2  lval1  rval1
0    a    x    0.0    6.0
1    b    y    1.0    NaN
2    c    z    2.0    8.0
3    b    a    NaN    7.0

In [27]:
left,right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
   key1 key2  rval1
 1    a    x      6
 2    b    a      7
 3    c    z      8)

In [28]:
# left join, merges all matched data, and only fills unmatched
# items from the left dataframe with NaN filled for the
# unmatched items in the result
# rows with labels 0 and 2
# match on key1 and key2 the row with label 1 is from left
left.merge(right, how='left')

  key1 key2  lval1  rval1
0    a    x      0    6.0
1    b    y      1    NaN
2    c    z      2    8.0

In [29]:
left,right

(  key1 key2  lval1
 0    a    x      0
 1    b    y      1
 2    c    z      2,
   key1 key2  rval1
 1    a    x      6
 2    b    a      7
 3    c    z      8)

In [30]:
# right join, merges all matched data, and only fills unmatched
# item from the right with NaN filled for the unmatched items
# in the result
# rows with labels 0 and 1 match on key1 and key2
# the row with label 2 is from right
left.merge(right, how='right')

  key1 key2  lval1  rval1
0    a    x    0.0      6
1    c    z    2.0      8
2    b    a    NaN      7

The pandas library also provides a .join() method that can be used to perform a join
using the index labels of the two DataFrame objects (instead of values in columns). Note
that if the columns in the two DataFrame objects do not have unique column names, you
must specify suffixes using the lsuffix and rsuffix parameters (automatic suffixing is
not performed). The following code demonstrates both the join and specification of
suffixes:

In [31]:
# join left with right (default method is outer)
# and since these DataFrame objects have duplicate column names
# we just specify lsuffix and rsuffix
left.join(right, lsuffix='_left', rsuffix='_right')

  key1_left key2_left  lval1 key1_right key2_right  rval1
0         a         x      0        NaN        NaN    NaN
1         b         y      1          a          x    6.0
2         c         z      2          b          a    7.0

The default type of join performed is an outer join. Note that this differs from the default
of the .merge() method, which defaults to inner. To change to an inner join, specify
how='inner', as is demonstrated in the following example:

In [32]:
# join left with right with an inner join
left.join(right, lsuffix='_left', rsuffix='_right', how='inner')

  key1_left key2_left  lval1 key1_right key2_right  rval1
1         b         y      1          a          x      6
2         c         z      2          b          a      7

<p>Notice that this is roughly equivalent to the earlier result from In[29] except with the
result having columns with slightly different names.</p><p>
It is also possible to perform right and left joins, but they lead to results similar to previous
examples, so they will be omitted for brevity.</p>

<h3>Pivoting</h3>

<p>Data is often stored in a stacked format, which is also referred to as record format; this is
common in databases, .csv files, and Excel spreadsheets. In a stacked format, the data is
often not normalized and has repeated values in many columns, or values that should
logically exists in other tables (violating another concept of tidy data).</p><p>Take the following data, which represents a stream of data from an accelerometer on a
mobile device (provided with the data from the sample code):</p>

In [39]:
# read in accellerometer data
sensor_readings = pd.read_csv("data/accel.csv")
sensor_readings

    interval axis  reading
0          0    X      0.0
1          0    Y      0.5
2          0    Z      1.0
3          1    X      0.1
4          1    Y      0.4
..       ...  ...      ...
7          2    Y      0.8
8          2    Z      0.3
9          3    X      0.2
10         3    Y      0.7
11         3    Z      0.2

[12 rows x 3 columns]

An issue with this data as it is organized is: how does one go about determining the
readings for a specific axis? This can be naively done with Boolean selections:

In [36]:
# extract X-axis readings
sensor_readings[sensor_readings['axis'] == 'X']

   interval axis  reading
0         0    X      0.0
3         1    X      0.1
6         2    X      0.3
9         3    X      0.2

<p>An issue here is what if you want to know the values for all axes at a given time, not just
the x axis? You can perform a selection for each value of the axis, but that is repetitive
code and does not handle the scenario of new axis values being inserted into DataFrame
without a change to the code.</p><p>A better representation would be where columns represent the unique variable values. To
convert to this form, use the DataFrame objects’ .pivot() function:</p>

In [40]:
sensor_readings

    interval axis  reading
0          0    X      0.0
1          0    Y      0.5
2          0    Z      1.0
3          1    X      0.1
4          1    Y      0.4
..       ...  ...      ...
7          2    Y      0.8
8          2    Z      0.3
9          3    X      0.2
10         3    Y      0.7
11         3    Z      0.2

[12 rows x 3 columns]

In [38]:
# pivot the data. Interval becomes the index, the columns are
# the current axes values, and use the readings as values
sensor_readings.pivot(index='interval',
columns='axis',
values='reading')

axis        X    Y    Z
interval               
0         0.0  0.5  1.0
1         0.1  0.4  0.2
2         0.3  0.8  0.3
3         0.2  0.7  0.2

This has taken all of the distinct values from the axis column, and pivoted them into
columns on the new DataFrame, while filling in values for the new columns from the
appropriate rows and columns of the original DataFrame. This new DataFrame
demonstrates that it is now very easy to identify the X, Y and Z sensor readings at each time
interval.

<h3>Example that applies in the next chapter of Stacking...</h3>

In [77]:
ak =sensor_readings.pivot(index='interval',
columns='axis',
values='reading')
ak.stack()

interval  axis
0         X       0.0
          Y       0.5
          Z       1.0
1         X       0.1
          Y       0.4
                 ... 
2         Y       0.8
          Z       0.3
3         X       0.2
          Y       0.7
          Z       0.2
Length: 12, dtype: float64

<h3>Stacking and unstacking</h3>

<p>Similar to the pivot function are the .stack() and .unstack() methods that are part of
both Series and DataFrame objects. The process of stacking pivots a level of column
labels to the row index. Unstacking performs the opposite, pivoting a level of the row
index into the column index</p><p>One of the differences between stacking/unstacking and performing a pivot is that unlike
pivots the stack and unstack functions will be able to pivot specific levels of a hierarchical
index. Also, where a pivot retains the same number of levels on an index, a stack and
unstack will always increase the levels on the index of one of the axes (columns for
unstack and rows for stack) and decrease the levels on the other axis.</p><p>The reasons for stacking and unstacking are along the same lines as for performing pivots.
Fundamentally it comes down to how you want your data organized for analysis. The
organization can change the means and ease of retrieving data and deriving results. As will
be demonstrated it also can have significant performance ramifications.</p><p>To understand the process of stacking and unstacking, we will first look at simpler cases
using nonhierarchical indexes, with very simple data, and focus on stacking. We then
progress to more complicated data using hierarchical indexes, revisiting the sensor data we
saw previously in the chapter and focusing on unstacking.</p>

In [69]:
# simple DataFrame with one column
df = pd.DataFrame({'a': [1, 2]}, index={'one', 'two'})
df

     a
one  1
two  2

In [71]:
# simple Series
ser = pd.Series([1, 2], index={'one', 'two'})
ser

one    1
two    2
dtype: int64

Stacking will move one level of the columns index into a new level of the rows index. As
our DataFrame only has one level, this collapses a DataFrame object into a Series object
with a hierarchical row index:

In [55]:
# push the column to another level of the index
# the result is a Series where values are looked up through
# a multi-index
stacked1 = df.stack()
stacked1

one  a    1
two  a    2
dtype: int64

In [57]:
stacked1.index

MultiIndex([('one', 'a'),
            ('two', 'a')],
           )

To access values, we now need to pass a tuple to the indexer of the Series object, which
does the lookup with just the index:

In [58]:
# lookup one / a using just the index via a tuple
stacked1[('one', 'a')]

1

If DataFrame contains multiple columns, then all of the columns are moved to the same
additional level of the new Series object:

In [60]:
# DataFrame with two columns
df = pd.DataFrame({'a': [1, 2], 'b': [3, 4]},
index={'one', 'two'})
df

     a  b
one  1  3
two  2  4

In [61]:
# push the two columns into a single level of the index
stacked2 = df.stack()
stacked2

one  a    1
     b    3
two  a    2
     b    4
dtype: int64

In [62]:
# lookup value with index of one / b
stacked2[('one', 'b')]

3

Unstacking will perform a similar operation in the opposite direction by moving a level of
the row index into a level of the columns axis. We will examine this process in the next
section as unstacking generally assumes that the index being unstacked is hierarchical.

In [78]:
stacked2

one  a    1
     b    3
two  a    2
     b    4
dtype: int64

In [79]:
stacked2.unstack()

     a  b
one  1  3
two  2  4

<h3>Unstacking using hierarchical indexes</h3>

To demonstrate unstacking with hierarchical indexes we will revisit the sensor data we
saw earlier in the chapter. However, we will add in an additional column to the
measurement data that represents readings for multiple users and copy data for two users.
The following sets up this data:

In [64]:
# make two copies of the sensor data, one for each user
user1 = sensor_readings.copy()
user2 = sensor_readings.copy()

# add names to the two copies
user1['who'] = 'Mike'
user2['who'] = 'Mikael'

user1,user2

(    interval axis  reading   who
 0          0    X      0.0  Mike
 1          0    Y      0.5  Mike
 2          0    Z      1.0  Mike
 3          1    X      0.1  Mike
 4          1    Y      0.4  Mike
 ..       ...  ...      ...   ...
 7          2    Y      0.8  Mike
 8          2    Z      0.3  Mike
 9          3    X      0.2  Mike
 10         3    Y      0.7  Mike
 11         3    Z      0.2  Mike
 
 [12 rows x 4 columns],
     interval axis  reading     who
 0          0    X      0.0  Mikael
 1          0    Y      0.5  Mikael
 2          0    Z      1.0  Mikael
 3          1    X      0.1  Mikael
 4          1    Y      0.4  Mikael
 ..       ...  ...      ...     ...
 7          2    Y      0.8  Mikael
 8          2    Z      0.3  Mikael
 9          3    X      0.2  Mikael
 10         3    Y      0.7  Mikael
 11         3    Z      0.2  Mikael
 
 [12 rows x 4 columns])

In [65]:
# for demonstration, let's scale user2's readings
user2['reading'] *= 100
user2

    interval axis  reading     who
0          0    X      0.0  Mikael
1          0    Y     50.0  Mikael
2          0    Z    100.0  Mikael
3          1    X     10.0  Mikael
4          1    Y     40.0  Mikael
..       ...  ...      ...     ...
7          2    Y     80.0  Mikael
8          2    Z     30.0  Mikael
9          3    X     20.0  Mikael
10         3    Y     70.0  Mikael
11         3    Z     20.0  Mikael

[12 rows x 4 columns]

In [81]:
# and reorganize this to have a hierarchical row index
ak = multi_user_sensor_data = pd.concat([user1, user2])
ak

    interval axis  reading     who
0          0    X      0.0    Mike
1          0    Y      0.5    Mike
2          0    Z      1.0    Mike
3          1    X      0.1    Mike
4          1    Y      0.4    Mike
..       ...  ...      ...     ...
7          2    Y     80.0  Mikael
8          2    Z     30.0  Mikael
9          3    X     20.0  Mikael
10         3    Y     70.0  Mikael
11         3    Z     20.0  Mikael

[24 rows x 4 columns]

In [82]:
multi_user_sensor_data = ak.set_index(['who', 'interval', 'axis'])
multi_user_sensor_data

                      reading
who    interval axis         
Mike   0        X         0.0
                Y         0.5
                Z         1.0
       1        X         0.1
                Y         0.4
...                       ...
Mikael 2        Y        80.0
                Z        30.0
       3        X        20.0
                Y        70.0
                Z        20.0

[24 rows x 1 columns]

With this organization in the data we can do things such as examine all the readings for a
specific person using just the index.

In [83]:
# look up user data for Mike using just the index
multi_user_sensor_data.loc['Mike']

               reading
interval axis         
0        X         0.0
         Y         0.5
         Z         1.0
1        X         0.1
         Y         0.4
...                ...
2        Y         0.8
         Z         0.3
3        X         0.2
         Y         0.7
         Z         0.2

[12 rows x 1 columns]

Or get all the readings of all axes and for all users at interval 1 using .xs().

In [84]:
# readings for all users and axes at interval 1
multi_user_sensor_data.xs(1, level='interval')

             reading
who    axis         
Mike   X         0.1
       Y         0.4
       Z         0.2
Mikael X        10.0
       Y        40.0
       Z        20.0

Unstacking will move the last level of the row index into a new level of the columns index
resulting in columns having MultiIndex. The following demonstrates the last level of this
unstacking (the axis level of the index):

In [85]:
multi_user_sensor_data

                      reading
who    interval axis         
Mike   0        X         0.0
                Y         0.5
                Z         1.0
       1        X         0.1
                Y         0.4
...                       ...
Mikael 2        Y        80.0
                Z        30.0
       3        X        20.0
                Y        70.0
                Z        20.0

[24 rows x 1 columns]

In [86]:
# unstack axis
multi_user_sensor_data.unstack()

                reading             
axis                  X     Y      Z
who    interval                     
Mikael 0            0.0  50.0  100.0
       1           10.0  40.0   20.0
       2           30.0  80.0   30.0
       3           20.0  70.0   20.0
Mike   0            0.0   0.5    1.0
       1            0.1   0.4    0.2
       2            0.3   0.8    0.3
       3            0.2   0.7    0.2

In [88]:
# unstack axis
multi_user_sensor_data.unstack().unstack()

         reading                          ...                               
axis           X                       Y  ...            Z                  
interval       0     1     2     3     0  ...     3      0     1     2     3
who                                       ...                               
Mikael       0.0  10.0  30.0  20.0  50.0  ...  70.0  100.0  20.0  30.0  20.0
Mike         0.0   0.1   0.3   0.2   0.5  ...   0.7    1.0   0.2   0.3   0.2

[2 rows x 12 columns]

In [89]:
# unstack axis
multi_user_sensor_data.unstack().unstack().unstack()

         axis  interval  who   
reading  X     0         Mikael     0.0
                         Mike       0.0
               1         Mikael    10.0
                         Mike       0.1
               2         Mikael    30.0
                                   ... 
         Z     1         Mike       0.2
               2         Mikael    30.0
                         Mike       0.3
               3         Mikael    20.0
                         Mike       0.2
Length: 24, dtype: float64

To unstack a different level use the level parameter. The following code unstacks the first
level (level=0):

In [91]:
multi_user_sensor_data

                      reading
who    interval axis         
Mike   0        X         0.0
                Y         0.5
                Z         1.0
       1        X         0.1
                Y         0.4
...                       ...
Mikael 2        Y        80.0
                Z        30.0
       3        X        20.0
                Y        70.0
                Z        20.0

[24 rows x 1 columns]

In [90]:
# unstack at level=0
multi_user_sensor_data.unstack(level=0)

              reading     
who            Mikael Mike
interval axis             
0        X        0.0  0.0
         Y       50.0  0.5
         Z      100.0  1.0
1        X       10.0  0.1
         Y       40.0  0.4
...               ...  ...
2        Y       80.0  0.8
         Z       30.0  0.3
3        X       20.0  0.2
         Y       70.0  0.7
         Z       20.0  0.2

[12 rows x 2 columns]

Multiple levels can be unstacked simultaneously by passing a list of the levels to
.unstack(). Additionally, if the levels are named, they can be specified by name instead
of location. The following unstacks the who and axis levels by name:

In [92]:
# unstack who and axis levels
unstacked = multi_user_sensor_data.unstack(['who', 'axis'])
unstacked

         reading                              
who         Mike           Mikael             
axis           X    Y    Z      X     Y      Z
interval                                      
0            0.0  0.5  1.0    0.0  50.0  100.0
1            0.1  0.4  0.2   10.0  40.0   20.0
2            0.3  0.8  0.3   30.0  80.0   30.0
3            0.2  0.7  0.2   20.0  70.0   20.0

To be thorough, we can restack this data. The following code will stack the who level of
the column back into the row index.

In [93]:
# and we can of course stack what we have unstacked
# this re-stacks who
unstacked.stack(level='who')

                reading             
axis                  X     Y      Z
interval who                        
0        Mikael     0.0  50.0  100.0
         Mike       0.0   0.5    1.0
1        Mikael    10.0  40.0   20.0
         Mike       0.1   0.4    0.2
2        Mikael    30.0  80.0   30.0
         Mike       0.3   0.8    0.3
3        Mikael    20.0  70.0   20.0
         Mike       0.2   0.7    0.2

<p>There are a couple of things worth pointing out about this result. First, stacking and
unstacking always move the levels into the last levels of the other index. Notice that the
who level is now the last level of the row index, but started out earlier as the first level.</p><p>This would have ramifications on the code to access elements via that index as it has
changed to another level. If you want to put a level back into another position you will
need to reorganize the indexes with other means than stacking and unstacking.</p><p>Second, with all this moving around of data, stacking and unstacking (as well as pivoting)
do not lose any information. They simply change the means by which it is organized and
accessed.</p>

<h3>Melting</h3>

<p>Melting is a type of unpivoting, and is often referred to as changing a DataFrame object
from wide format to long format. This format is common in various statistical analyses,
and data you read may be provided already in a melted form, or you may need to pass data
in this format to other code that expects this organization.</p>

Technically, melting is the process of reshaping a DataFrame into a format where two or
more columns, referred to as variable and value, are created by unpivoting column
labels in the variable column, and then moving the data from these columns into the
appropriate location in the value column. All other columns are then made into identifier
columns that assist in describing the data.

In [97]:
# we will demonstrate melting with this DataFrame
data = pd.DataFrame({'Name' : ['Mike', 'Mikael'], 'Height' : [6.1, 6.0], 'Weight' : [220, 185], 'Age': [23, 28]})
data

     Name  Height  Weight  Age
0    Mike     6.1     220   23
1  Mikael     6.0     185   28

The following melts this DataFrame, using the Name column as the identifier column, and
the Height and Weight columns as measured variables. The Name column remains, with
the Height and Weight columns unpivoted into the variable column. Then the values
from these two columns are rearranged into the value column, and ensured to align with
the appropriate combination values of Name and variable that would have existed in the
original data:

In [100]:
# melt it, use Name as the id,
# Height and Weight columns as the variables
pd.melt(data, id_vars=['Name'], value_vars=['Height', 'Weight'])

     Name variable  value
0    Mike   Height    6.1
1  Mikael   Height    6.0
2    Mike   Weight  220.0
3  Mikael   Weight  185.0

The data is now restructured so that it is easy to extract the value for any combination of
variable and Name. Additionally, when in this format it is easier to add a new variable and
measurement as the data can simply be added as a new row instead of requiring a change
of structure to DataFrame by adding a new column.

<h3>Performance benefits of stacked data</h3>

Finally, we will examine a reason for which we would want to stack data like this. This is
because it can be shown to be more efficient than using lookup through a single level
index and then a column lookup, or even compared to an .iloc lookup, specifying the
location of the row and column by location. The following demonstrates this:

In [101]:
# time the different methods
import timeit

t = timeit.Timer("stacked1[('one', 'a')]", "from __main__ import stacked1, df")
r1 = timeit.timeit(lambda: stacked1.loc[('one', 'a')], number=10000)
r2 = timeit.timeit(lambda: df.loc['one']['a'], number=10000)
r3 = timeit.timeit(lambda: df.iloc[1, 0], number=10000)

In [102]:
# and the results are… Yes, it's the fastest of the three
r1, r2, r3

(0.44381511099709314, 1.311863232000178, 0.08189404600125272)

This can have extreme benefits for application performance if we need to repeatedly
access a large number of scalar values out of a DataFrame.